In [17]:
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST
from neuralforecast.losses.pytorch import DistributionLoss
from data_loading_finance.loading_data import LoadingData
from ts_tools.tools import tools

In [8]:
logging.getLogger("prophet").setLevel(logging.ERROR)
logging.getLogger("cmdstanpy").disabled = True

In [11]:
# Cargar datos y preprocesarlos
tickers = ['NVDA']
loader = LoadingData(tickers=tickers)
ts = loader.get_data(start_date='2015-01-01', interval='1mo')
info = loader.get_info_ticker()

ts_tools = tools()
ts = ts_tools.ts_prepartion(ts, 'Date', 'Adj Close')

[*********************100%%**********************]  1 of 1 completed

Getting info for ticker NVDA...


In [13]:
# Split data into training and testing sets
train_size = int(len(ts) * 0.8)
ts_train, ts_test = ts[:train_size], ts[train_size:]

# Create DataFrames for NeuralForecast
Y_train_df = ts_train.copy()
Y_test_df = ts_test.copy()

In [15]:
model = PatchTST(h=12,
                 input_size=104,
                 patch_len=24,
                 stride=24,
                 revin=False,
                 hidden_size=16,
                 n_heads=4,
                 scaler_type='robust',
                 loss=DistributionLoss(distribution='StudentT', level=[80, 90]),
                 learning_rate=1e-3,
                 max_steps=500,
                 val_check_steps=50,
                 early_stop_patience_steps=2)

nf = NeuralForecast(
    models=[model],
    freq='M'
)

Seed set to 1


In [16]:
# Fit the model
nf.fit(df=Y_train_df, static_df=None, val_size=12)

# Make predictions
forecasts = nf.predict(futr_df=Y_test_df)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type              | Params | Mode 
-----------------------------------------------------------
0 | loss         | DistributionLoss  | 5      | train
1 | padder_train | ConstantPad1d     | 0      | train
2 | scaler       | TemporalNorm      | 0      | train
3 | model        | PatchTST_backbone | 32.2 K | train
-----------------------------------------------------------
32.2 K    Trainable params
8         Non-trainable params
32.3 K    Total params
0.129     Total estimated model params size (MB)


Exception: Could not deserialize ATN with version  (expected 4).